In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

import numpy as np
import pandas as pd
import random

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from sympy.geometry import Point3D

In [2]:
file_folder = '../../data/input'
os.listdir(file_folder)

['sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'mulliken_charges.csv',
 'train.csv',
 'test.csv',
 'structures.csv',
 'structures']

In [3]:
train = pd.read_csv(f'{file_folder}/train.csv')
test = pd.read_csv(f'{file_folder}/test.csv')
magnetic_shielding_tensors = pd.read_csv(f'{file_folder}/magnetic_shielding_tensors.csv')
dipole_moments = pd.read_csv(f'{file_folder}/dipole_moments.csv')
mulliken_charges = pd.read_csv(f'{file_folder}/mulliken_charges.csv')
potential_energy = pd.read_csv(f'{file_folder}/potential_energy.csv')
scalar_coupling_contributions = pd.read_csv(f'{file_folder}/scalar_coupling_contributions.csv')
structures = pd.read_csv(f'{file_folder}/structures.csv')

In [4]:
import gc
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn import metrics
# Graph Stuff
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [21]:
df = pd.read_csv(f'{file_folder}/train.csv')
df.atom_index_0+=1
df.atom_index_1+=1

In [22]:
df_test = pd.read_csv(f'{file_folder}/test.csv')
df_test.atom_index_0+=1
df_test.atom_index_1+=1

In [23]:
trainmolecules = df.molecule_name.unique()
testmolecules = df_test.molecule_name.unique()

In [24]:
le = LabelEncoder()
le.fit(df.type.tolist() + df_test.type.tolist())
df.type= le.transform(df.type) + 1
df_test.type= le.transform(df_test.type)+1

In [25]:
df_structures = pd.read_csv(f'{file_folder}/structures.csv')
df_structures.atom_index+=1
df_structures.head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,1,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,2,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,3,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,4,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,5,H,-0.523814,1.437933,0.906397


In [26]:
le = LabelEncoder()
df_structures.atom = le.fit_transform(df_structures.atom)+1

In [27]:
df_structures.head(20)

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,1,1,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,2,3,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,3,3,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,4,3,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,5,3,-0.523814,1.437933,0.906397
5,dsgdb9nsd_000002,1,4,-0.040426,1.024108,0.062564
6,dsgdb9nsd_000002,2,3,0.017257,0.012545,-0.027377
7,dsgdb9nsd_000002,3,3,0.915789,1.358745,-0.028758
8,dsgdb9nsd_000002,4,3,-0.520278,1.343532,-0.775543
9,dsgdb9nsd_000003,1,5,-0.034360,0.977540,0.007602


In [12]:
z = None
for i in np.arange(0,5,1):
    print(i)
    if(z is None):
        z = df[['id','molecule_name','atom_index_0','atom_index_1','type','scalar_coupling_constant']].copy()
    y = df_structures.copy()
    y.atom_index -= i
   
    z = z.merge(y, how='left',left_on=['molecule_name','atom_index_0'], right_on=['molecule_name','atom_index'],suffixes=('', '_neg_'+str(i)))
    if(z.columns.contains('atom_index_neg_'+str(i))):
        del z['atom_index_neg_'+str(i)]

    if(i!=0):
        z['dist_neg_'+str(i)] = 1./np.linalg.norm(z[['x', 'y', 'z']].values - z[['x_neg_'+str(i), 'y_neg_'+str(i), 'z_neg_'+str(i)]].values, axis=1)
        del z['x_neg_'+str(i)]
        del z['y_neg_'+str(i)]
        del z['z_neg_'+str(i)]

    
for i in np.arange(1,5,1):
    print(i)
    
    y = df_structures.copy()
    y.atom_index += i

    
    z = z.merge(y, how='left',left_on=['molecule_name','atom_index_0'], right_on=['molecule_name','atom_index'],suffixes=('', '_pos_'+str(i)))    
    if(z.columns.contains('atom_index')):
        del z['atom_index']
    if(z.columns.contains('atom_index_pos_'+str(i))):
        del z['atom_index_pos_'+str(i)]
    z['dist_pos_'+str(i)] = 1./np.linalg.norm(z[['x', 'y', 'z']].values - z[['x_pos_'+str(i), 'y_pos_'+str(i), 'z_pos_'+str(i)]].values, axis=1)
    del z['x_pos_'+str(i)]
    del z['y_pos_'+str(i)]
    del z['z_pos_'+str(i)]
del z['x']
del z['y']
del z['z']    

for i in np.arange(0,5,1):
    print(i)
    y = df_structures.copy()
    y.atom_index -= i
   
    z = z.merge(y, how='left',left_on=['molecule_name','atom_index_1'], right_on=['molecule_name','atom_index'],suffixes=('', '_1_neg_'+str(i)))
    if(z.columns.contains('atom_index_1_neg_'+str(i))):
        del z['atom_index_1_neg_'+str(i)]

    if(i!=0):
        z['dist_1_neg_'+str(i)] = 1./np.linalg.norm(z[['x', 'y', 'z']].values - z[['x_1_neg_'+str(i), 'y_1_neg_'+str(i), 'z_1_neg_'+str(i)]].values, axis=1)
        del z['x_1_neg_'+str(i)]
        del z['y_1_neg_'+str(i)]
        del z['z_1_neg_'+str(i)]

    
for i in np.arange(1,5,1):
    print(i)

    y = df_structures.copy()
    y.atom_index += i
   
    z = z.merge(y, how='left',left_on=['molecule_name','atom_index_1'], right_on=['molecule_name','atom_index'],suffixes=('', '_1_pos_'+str(i)))    
    if(z.columns.contains('atom_index')):
        del z['atom_index']
    if(z.columns.contains('atom_index_1_pos_'+str(i))):
        del z['atom_index_1_pos_'+str(i)]
    z['dist_1_pos_'+str(i)] = 1./np.linalg.norm(z[['x', 'y', 'z']].values - z[['x_1_pos_'+str(i), 'y_1_pos_'+str(i), 'z_1_pos_'+str(i)]].values, axis=1)
    del z['x_1_pos_'+str(i)]
    del z['y_1_pos_'+str(i)]
    del z['z_1_pos_'+str(i)]    
  
del z['x']
del z['y']
del z['z']

0


/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning:

The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'



1
2
3
4
1


/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/ipykernel/__main__.py:28: FutureWarning:

The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/ipykernel/__main__.py:30: FutureWarning:

The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'



2
3
4
0


/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/ipykernel/__main__.py:46: FutureWarning:

The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'



1
2
3
4
1


/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/ipykernel/__main__.py:63: FutureWarning:

The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/ipykernel/__main__.py:65: FutureWarning:

The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'



2
3
4


In [13]:
z.shape

(4658147, 40)

In [14]:
import copy

In [15]:
df_train = copy.deepcopy(z)

In [16]:
'id' in df_train.columns

True

In [28]:
z = None
for i in np.arange(0,5,1):
    print(i)
    if(z is None):
        z = df_test[['id','molecule_name','atom_index_0','atom_index_1','type']].copy()
    y = df_structures.copy()
    y.atom_index -= i
   
    z = z.merge(y, how='left',left_on=['molecule_name','atom_index_0'], right_on=['molecule_name','atom_index'],suffixes=('', '_neg_'+str(i)))
    if(z.columns.contains('atom_index_neg_'+str(i))):
        del z['atom_index_neg_'+str(i)]

    if(i!=0):
        z['dist_neg_'+str(i)] = 1./np.linalg.norm(z[['x', 'y', 'z']].values - z[['x_neg_'+str(i), 'y_neg_'+str(i), 'z_neg_'+str(i)]].values, axis=1)
        del z['x_neg_'+str(i)]
        del z['y_neg_'+str(i)]
        del z['z_neg_'+str(i)]

    
for i in np.arange(1,5,1):
    print(i)
    
    y = df_structures.copy()
    y.atom_index += i

    
    z = z.merge(y, how='left',left_on=['molecule_name','atom_index_0'], right_on=['molecule_name','atom_index'],suffixes=('', '_pos_'+str(i)))    
    if(z.columns.contains('atom_index')):
        del z['atom_index']
    if(z.columns.contains('atom_index_pos_'+str(i))):
        del z['atom_index_pos_'+str(i)]
    z['dist_pos_'+str(i)] = 1./np.linalg.norm(z[['x', 'y', 'z']].values - z[['x_pos_'+str(i), 'y_pos_'+str(i), 'z_pos_'+str(i)]].values, axis=1)
    del z['x_pos_'+str(i)]
    del z['y_pos_'+str(i)]
    del z['z_pos_'+str(i)]
del z['x']
del z['y']
del z['z']    

for i in np.arange(0,5,1):
    print(i)
    y = df_structures.copy()
    y.atom_index -= i
   
    z = z.merge(y, how='left',left_on=['molecule_name','atom_index_1'], right_on=['molecule_name','atom_index'],suffixes=('', '_1_neg_'+str(i)))
    if(z.columns.contains('atom_index_1_neg_'+str(i))):
        del z['atom_index_1_neg_'+str(i)]

    if(i!=0):
        z['dist_1_neg_'+str(i)] = 1./np.linalg.norm(z[['x', 'y', 'z']].values - z[['x_1_neg_'+str(i), 'y_1_neg_'+str(i), 'z_1_neg_'+str(i)]].values, axis=1)
        del z['x_1_neg_'+str(i)]
        del z['y_1_neg_'+str(i)]
        del z['z_1_neg_'+str(i)]

    
for i in np.arange(1,5,1):
    print(i)

    y = df_structures.copy()
    y.atom_index += i
   
    z = z.merge(y, how='left',left_on=['molecule_name','atom_index_1'], right_on=['molecule_name','atom_index'],suffixes=('', '_1_pos_'+str(i)))    
    if(z.columns.contains('atom_index')):
        del z['atom_index']
    if(z.columns.contains('atom_index_1_pos_'+str(i))):
        del z['atom_index_1_pos_'+str(i)]
    z['dist_1_pos_'+str(i)] = 1./np.linalg.norm(z[['x', 'y', 'z']].values - z[['x_1_pos_'+str(i), 'y_1_pos_'+str(i), 'z_1_pos_'+str(i)]].values, axis=1)
    del z['x_1_pos_'+str(i)]
    del z['y_1_pos_'+str(i)]
    del z['z_1_pos_'+str(i)]    
  
del z['x']
del z['y']
del z['z']

0


/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning:

The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'



1
2
3
4
1


/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/ipykernel/__main__.py:28: FutureWarning:

The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/ipykernel/__main__.py:30: FutureWarning:

The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'



2
3
4
0


/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/ipykernel/__main__.py:46: FutureWarning:

The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'



1
2
3
4
1


/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/ipykernel/__main__.py:63: FutureWarning:

The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/ipykernel/__main__.py:65: FutureWarning:

The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'



2
3
4


In [29]:
df_test = copy.deepcopy(z)

In [30]:
df_test.shape

(2505542, 39)

In [31]:
df_train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom,atom_neg_1,dist_neg_1,atom_neg_2,...,atom_1_neg_4,dist_1_neg_4,atom_1_pos_1,dist_1_pos_1,atom_1_pos_2,dist_1_pos_2,atom_1_pos_3,dist_1_pos_3,atom_1_pos_4,dist_1_pos_4
0,0,dsgdb9nsd_000001,2,1,1,84.8076,3,3.0,0.560815,3.0,...,3.0,0.915795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,dsgdb9nsd_000001,2,3,4,-11.2570,3,3.0,0.560815,3.0,...,NaN,NaN,3.0,0.560815,1.0,0.915791,NaN,NaN,NaN,NaN
2,2,dsgdb9nsd_000001,2,4,4,-11.2548,3,3.0,0.560815,3.0,...,NaN,NaN,3.0,0.560803,3.0,0.560806,1.0,0.915796,NaN,NaN
3,3,dsgdb9nsd_000001,2,5,4,-11.2543,3,3.0,0.560815,3.0,...,NaN,NaN,3.0,0.560806,3.0,0.560806,3.0,0.560803,1.0,0.915795
4,4,dsgdb9nsd_000001,3,1,1,84.8074,3,3.0,0.560803,3.0,...,3.0,0.915795,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [33]:
df_train = reduce_mem_usage(df_train)

Mem. usage decreased to 404.25 Mb (72.3% reduction)


In [34]:
df_test = reduce_mem_usage(df_test)

Mem. usage decreased to 210.27 Mb (72.5% reduction)


In [35]:
df_train.drop(columns=['molecule_name','atom_index_0','atom_index_1','type','scalar_coupling_constant']).to_pickle('../../data/feature/stanislav-blinov_train.pkl')

In [36]:
df_test.drop(columns=['molecule_name','atom_index_0','atom_index_1','type']).to_pickle('../../data/feature/stanislav-blinov_test.pkl')